In [1]:
#!/usr/bin/env python

'''
This script will do the delaunay tessellation of input 3D coorinates and returns the tetrahedraliy 
histogram


usage "python DelaunayTetra.py xx.pdb xx.traj up"
'''

import string
import struct
import sys
import numpy as np
from MDAnalysis import *
import MDAnalysis
import MDAnalysis.lib.distances
import time
import numpy.linalg
import scipy.stats
import matplotlib.pyplot as plt
import math
import MDAnalysis.lib.NeighborSearch as NS
from scipy.spatial import Voronoi, voronoi_plot_2d, SphericalVoronoi, Delaunay
import multiprocessing as mp
import pandas as pd
import seaborn as sb

In [2]:
def GetCoords(ts):
    #print ('Frame Coords %i'%(ts))
    u = MDAnalysis.Universe(top,traj)
    u.trajectory[ts]
    x_box = u.dimensions[0]
    y_box = u.dimensions[1]
    z_box = u.dimensions[2]

    # Select atoms within this particular frame
    atoms1 = u.select_atoms(sel1)
    atoms2 = u.select_atoms(sel2)
    atoms3 = u.select_atoms(sel3)
    
    
    #print (atoms1.positions)
    zmean = np.mean(np.concatenate([atoms1.positions[:,2],atoms2.positions[:,2], atoms3.positions[:,2]]))
 
    num_lpd1 = atoms1.n_atoms
    num_lpd2 = atoms2.n_atoms
    num_lpd3 = atoms3.n_atoms

    if side == 'up':
    
        lpd1i=[]
        zpos = atoms1.positions[:,2] 
        for i in range(num_lpd1):
            if zpos[i] > zmean:
                lpd1i.append(atoms1[i])
        lpd1i = np.sum(np.array(lpd1i))

        
        lpd2i=[]
        pos = atoms2.positions[:,2] 
        for i in range(num_lpd2):
            if zpos[i] > zmean:
                lpd2i.append(atoms2[i])
        lpd2i = np.sum(np.array(lpd2i))

        lipids = lpd1i + lpd2i
        #print(lipids.positions)

        u.trajectory[ts] # now move to the frame currently being analyzed
        ns_lipids = NS.AtomNeighborSearch(atoms3)

        lpd3i = ns_lipids.search(lipids,10.0) 

    
    if side == 'down': 
        lpd1i=[]
        zpos = atoms1.positions[:,2]
        
        for i in range(num_lpd1):
            if zpos[i] < zmean:
                lpd1i.append(atoms1[i])
        lpd1i = np.sum(np.array(lpd1i))

        lpd2i=[]
        for i in range(num_lpd2):
            if zpos[i] < zmean:
                lpd2i.append(atoms2[i])
        lpd2i = np.sum(np.array(lpd2i))

        lipids = lpd1i + lpd2i
        #print(lipids.positions)

        #u.trajectory[ts] # now move to the frame currently being analyzed
        ns_lipids = NS.AtomNeighborSearch(atoms3)

        lpd3i = ns_lipids.search(lipids,10.0) 
        
    if side == 'none': 
        lpd1i=[]
        zpos = atoms1.positions[:,2]
        
        for i in range(num_lpd1):
            if ((zpos[i] < zmean) or (zpos[i] > zmean)):
                lpd1i.append(atoms1[i])
        lpd1i = np.sum(np.array(lpd1i))

        lpd2i=[]
        for i in range(num_lpd2):
            if ((zpos[i] < zmean) or (zpos[i] > zmean)):
                lpd2i.append(atoms2[i])
        lpd2i = np.sum(np.array(lpd2i))

        lipids = lpd1i + lpd2i
        #print(lipids.positions)

        #u.trajectory[ts] # now move to the frame currently being analyzed
        ns_lipids = NS.AtomNeighborSearch(atoms3)

        lpd3i = ns_lipids.search(lipids,10.0) 
    
    lipids = lpd1i + lpd2i + lpd3i
           
    #######Extracting XYZ coordinates

    POS = lipids.positions
    Pxyz= []
    if (len(POS) == 0):
        print ("ERROR!!!1-Zero Length Array\n Not Possible")
        exit(1)
    else:    
        for l in range(0,len(POS)) :
            Pxyz.append([POS[l][0],POS[l][1], POS[l][2]])    

    #u.trajectory[ts] # now move to the frame currently being analyzed

    atm_list=[]
    for l in range(0,len(Pxyz)) :
        atm_list.append([Pxyz[l][0],Pxyz[l][1], Pxyz[l][2], lipids[l].resname, lipids[l].resid])

        
    #Introducing PBC
    x_box = u.dimensions[0]
    y_box = u.dimensions[1]

    xplus   = []
    xminus  = []
    xyplus  = []
    xyminus = []

    for atm in range(0 ,len(atm_list)):
        xplus.append([atm_list[atm][0]+x_box,atm_list[atm][1],atm_list[atm][2]])
        xminus.append([atm_list[atm][0]-x_box,atm_list[atm][1],atm_list[atm][2]])

    atm_list_px = atm_list + xplus + xminus

    for atm in range(0 ,len(atm_list_px)):
        xyplus.append([atm_list_px[atm][0],atm_list_px[atm][1]+y_box,atm_list_px[atm][2]])
        xyminus.append([atm_list_px[atm][0],atm_list_px[atm][1]-y_box,atm_list_px[atm][2]])

    atm_list_p = atm_list_px + xyplus + xyminus


    atm_xy = []
    for i in range(0,len(atm_list_p)) :
        atm_xy.append([atm_list_p[i][0],atm_list_p[i][1]])  
        
    return (Pxyz, atm_list, atm_xy)    

In [3]:
def EdgeLength(A,B):
    dist=math.sqrt( ((float(A[0]) -float(B[0]))**2 ) +  ((float(A[1]) -float(B[1]))**2 ) + ((float(A[2]) -float(B[2]))**2 )   )
    return dist

In [4]:

def Delaunay_Tessellation(ts):
    print ('Frame %i in %i'%(ts, end_f))
    points, atm_list, atm_xy=GetCoords(ts) 
    tri=Delaunay(points)   
    
    #plt.plot(points[:, 0], points[:, 1], points[:, 2], c='b', linestyle='dashed')
    #plt.plot(sv.vertices[:, 0], sv.vertices[:, 1], sv.vertices[:, 2], c='g')
    #plt.show()
    #print ("shape", tri.simplices.shape)
    #print ("#",tri.simplices)
    #tri.simplices[0]
    #print ("#",tri.neighbors)
    #points[tri.simplices]

    ridge_points   = tri.simplices
    r_length       = len(ridge_points)
    tetrahedrality = []
    for k in range(0, r_length):
        
        sum_edge       = 0
        sumd           = 0
        tetra          = 0
        lbar           = 0
        ridge_k = ridge_points[k]
        L0 = points[int(ridge_k[0])]
        L1 = points[int(ridge_k[1])] 
        L2 = points[int(ridge_k[2])] 
        L3 = points[int(ridge_k[3])]   
        #print (ridge_k, L0, L1, L2, L3) 
        
        L01 = EdgeLength(L0,L1)
        L02 = EdgeLength(L0,L2)
        L03 = EdgeLength(L0,L3)
        L12 = EdgeLength(L1,L2)
        L13 = EdgeLength(L1,L3)
        L23 = EdgeLength(L2,L3)
        
        sumd += L01 + L02 + L03 + L12 + L13 + L23 
        lbar= sumd/15.0
        
        a1  = EdgeLength(L0,L1) - EdgeLength(L0,L2)
        a2  = EdgeLength(L0,L1) - EdgeLength(L0,L3)
        a3  = EdgeLength(L0,L1) - EdgeLength(L1,L2)
        a4  = EdgeLength(L0,L1) - EdgeLength(L1,L3)
        a5  = EdgeLength(L0,L1) - EdgeLength(L2,L3)
        
        a6  = EdgeLength(L0,L2) - EdgeLength(L0,L3)
        a7  = EdgeLength(L0,L2) - EdgeLength(L1,L2)
        a8  = EdgeLength(L0,L2) - EdgeLength(L1,L3)
        a9  = EdgeLength(L0,L2) - EdgeLength(L2,L3)
        
        a10 = EdgeLength(L0,L3) - EdgeLength(L1,L2)
        a11 = EdgeLength(L0,L3) - EdgeLength(L1,L2)
        a12 = EdgeLength(L0,L3) - EdgeLength(L1,L2)
        
        a13 = EdgeLength(L1,L2) - EdgeLength(L1,L3)
        a14 = EdgeLength(L1,L2) - EdgeLength(L2,L3)
        
        a15 = EdgeLength(L1,L3) - EdgeLength(L2,L3)
        
        
        sum_edge += a1*a1 + a2*a2 + a3*a3 + a4*a4 + a5*a5 + a6*a6 + a7*a7 + a8*a8 + a9*a9 + a10*a10  + a11*a11 + a12*a12 + a13*a13 + a14*a14 + a15*a15    
        
        tetra = sum_edge/(15*lbar*lbar) 
        #print ("lbar", lbar, "sumd", sumd, "sum_edge", sum_edge, "tet", tetra)
        tetrahedrality.append(tetra) 
        
    return (tetrahedrality)



def Histogram(data):
    mat   = np.asarray(pd.DataFrame(data)).flatten()
    mat1 = mat[numpy.logical_not(numpy.isnan(mat))]
    np.savetxt('Tetra_10CAPE.dat', mat1, header="CAPE")
    ax=sb.distplot(mat1, hist=True, kde=True, color="r", label="CAPE", hist_kws={ "alpha":1, "linewidth": 3})
    ax.set_ylabel("Probability")
    ax.set_xlabel("T")
    ax.set_title("Tetrahedrality", fontsize='large')
    ax.legend()
    plt.savefig("Tetrahedrality_Popc_10CAPE.png", dpi=600, facecolor='w', edgecolor='w',orientation='portrait')   
    #plt.show()



In [6]:
from MDAnalysis.tests.datafiles import PDB
import MDAnalysis.lib.NeighborSearch as NS

if __name__ == "__main__":
    print  ('Initiating Voroni Tesselation')
    #top  =sys.argv[1]
    #traj =sys.argv[2]
    #side =sys.argv[3] # "up" for upper leaflet "down" for lower leaflet

    top  = "POPC_CHL_10CAPE_Production_1.gro"
    traj = "POPC_CHL_10CAPE_Production_1_599_nojump.xtc"
    side = "none"
   
    u = MDAnalysis.Universe(top,traj)
    
    #Lipid Residue names
    lipid1 ='POPC'
    lipid2 ='CHL1'
    lipid3 ='TIP3'

    # Atom selections
    sel1 = 'resname %s and name P'%lipid1
    sel2 = 'resname %s and name O3'%lipid2
    sel3 = 'resname %s and name OH2'%lipid3
 
    #Frames to be calculated
    end_f = u.trajectory.n_frames

    start_f = 0
    skip    = 1
    end_f   = 5000  
   

    # Number of processors to use in multiprocessing
    nprocs = 1

    frames = np.arange(start_f, end_f)[::skip]
    n_frames = len(frames)
    print (n_frames)
     
    pool = mp.Pool(processes=nprocs)
    
    print ('Initiating multiprocessing with %i processors'%nprocs)
    
    Tetra=pool.map(Delaunay_Tessellation,frames) 
    
    Histogram(Tetra)
    

Initiating Voroni Tesselation
5000
Frame 0 in 5000
Initiating multiprocessing with 1 processors
Frame 1 in 5000
Frame 2 in 5000
Frame 3 in 5000
Frame 4 in 5000
Frame 5 in 5000
Frame 6 in 5000
Frame 7 in 5000
Frame 8 in 5000
Frame 9 in 5000
Frame 10 in 5000
Frame 11 in 5000
Frame 12 in 5000
Frame 13 in 5000
Frame 14 in 5000
Frame 15 in 5000
Frame 16 in 5000
Frame 17 in 5000
Frame 18 in 5000
Frame 19 in 5000
Frame 20 in 5000
Frame 21 in 5000
Frame 22 in 5000
Frame 23 in 5000
Frame 24 in 5000
Frame 25 in 5000
Frame 26 in 5000
Frame 27 in 5000
Frame 28 in 5000
Frame 29 in 5000
Frame 30 in 5000
Frame 31 in 5000
Frame 32 in 5000
Frame 33 in 5000
Frame 34 in 5000
Frame 35 in 5000
Frame 36 in 5000
Frame 37 in 5000
Frame 38 in 5000
Frame 39 in 5000
Frame 40 in 5000
Frame 41 in 5000
Frame 42 in 5000
Frame 43 in 5000
Frame 44 in 5000
Frame 45 in 5000
Frame 46 in 5000
Frame 47 in 5000
Frame 48 in 5000
Frame 49 in 5000
Frame 50 in 5000
Frame 51 in 5000
Frame 52 in 5000
Frame 53 in 5000
Frame 54 in 

Frame 457 in 5000
Frame 458 in 5000
Frame 459 in 5000
Frame 460 in 5000
Frame 461 in 5000
Frame 462 in 5000
Frame 463 in 5000
Frame 464 in 5000
Frame 465 in 5000
Frame 466 in 5000
Frame 467 in 5000
Frame 468 in 5000
Frame 469 in 5000
Frame 470 in 5000
Frame 471 in 5000
Frame 472 in 5000
Frame 473 in 5000
Frame 474 in 5000
Frame 475 in 5000
Frame 476 in 5000
Frame 477 in 5000
Frame 478 in 5000
Frame 479 in 5000
Frame 480 in 5000
Frame 481 in 5000
Frame 482 in 5000
Frame 483 in 5000
Frame 484 in 5000
Frame 485 in 5000
Frame 486 in 5000
Frame 487 in 5000
Frame 488 in 5000
Frame 489 in 5000
Frame 490 in 5000
Frame 491 in 5000
Frame 492 in 5000
Frame 493 in 5000
Frame 494 in 5000
Frame 495 in 5000
Frame 496 in 5000
Frame 497 in 5000
Frame 498 in 5000
Frame 499 in 5000
Frame 500 in 5000
Frame 501 in 5000
Frame 502 in 5000
Frame 503 in 5000
Frame 504 in 5000
Frame 505 in 5000
Frame 506 in 5000
Frame 507 in 5000
Frame 508 in 5000
Frame 509 in 5000
Frame 510 in 5000
Frame 511 in 5000
Frame 512 

Frame 913 in 5000
Frame 914 in 5000
Frame 915 in 5000
Frame 916 in 5000
Frame 917 in 5000
Frame 918 in 5000
Frame 919 in 5000
Frame 920 in 5000
Frame 921 in 5000
Frame 922 in 5000
Frame 923 in 5000
Frame 924 in 5000
Frame 925 in 5000
Frame 926 in 5000
Frame 927 in 5000
Frame 928 in 5000
Frame 929 in 5000
Frame 930 in 5000
Frame 931 in 5000
Frame 932 in 5000
Frame 933 in 5000
Frame 934 in 5000
Frame 935 in 5000
Frame 936 in 5000
Frame 937 in 5000
Frame 938 in 5000
Frame 939 in 5000
Frame 940 in 5000
Frame 941 in 5000
Frame 942 in 5000
Frame 943 in 5000
Frame 944 in 5000
Frame 945 in 5000
Frame 946 in 5000
Frame 947 in 5000
Frame 948 in 5000
Frame 949 in 5000
Frame 950 in 5000
Frame 951 in 5000
Frame 952 in 5000
Frame 953 in 5000
Frame 954 in 5000
Frame 955 in 5000
Frame 956 in 5000
Frame 957 in 5000
Frame 958 in 5000
Frame 959 in 5000
Frame 960 in 5000
Frame 961 in 5000
Frame 962 in 5000
Frame 963 in 5000
Frame 964 in 5000
Frame 965 in 5000
Frame 966 in 5000
Frame 967 in 5000
Frame 968 

Frame 1349 in 5000
Frame 1350 in 5000
Frame 1351 in 5000
Frame 1352 in 5000
Frame 1353 in 5000
Frame 1354 in 5000
Frame 1355 in 5000
Frame 1356 in 5000
Frame 1357 in 5000
Frame 1358 in 5000
Frame 1359 in 5000
Frame 1360 in 5000
Frame 1361 in 5000
Frame 1362 in 5000
Frame 1363 in 5000
Frame 1364 in 5000
Frame 1365 in 5000
Frame 1366 in 5000
Frame 1367 in 5000
Frame 1368 in 5000
Frame 1369 in 5000
Frame 1370 in 5000
Frame 1371 in 5000
Frame 1372 in 5000
Frame 1373 in 5000
Frame 1374 in 5000
Frame 1375 in 5000
Frame 1376 in 5000
Frame 1377 in 5000
Frame 1378 in 5000
Frame 1379 in 5000
Frame 1380 in 5000
Frame 1381 in 5000
Frame 1382 in 5000
Frame 1383 in 5000
Frame 1384 in 5000
Frame 1385 in 5000
Frame 1386 in 5000
Frame 1387 in 5000
Frame 1388 in 5000
Frame 1389 in 5000
Frame 1390 in 5000
Frame 1391 in 5000
Frame 1392 in 5000
Frame 1393 in 5000
Frame 1394 in 5000
Frame 1395 in 5000
Frame 1396 in 5000
Frame 1397 in 5000
Frame 1398 in 5000
Frame 1399 in 5000
Frame 1400 in 5000
Frame 1401 i

Frame 1781 in 5000
Frame 1782 in 5000
Frame 1783 in 5000
Frame 1784 in 5000
Frame 1785 in 5000
Frame 1786 in 5000
Frame 1787 in 5000
Frame 1788 in 5000
Frame 1789 in 5000
Frame 1790 in 5000
Frame 1791 in 5000
Frame 1792 in 5000
Frame 1793 in 5000
Frame 1794 in 5000
Frame 1795 in 5000
Frame 1796 in 5000
Frame 1797 in 5000
Frame 1798 in 5000
Frame 1799 in 5000
Frame 1800 in 5000
Frame 1801 in 5000
Frame 1802 in 5000
Frame 1803 in 5000
Frame 1804 in 5000
Frame 1805 in 5000
Frame 1806 in 5000
Frame 1807 in 5000
Frame 1808 in 5000
Frame 1809 in 5000
Frame 1810 in 5000
Frame 1811 in 5000
Frame 1812 in 5000
Frame 1813 in 5000
Frame 1814 in 5000
Frame 1815 in 5000
Frame 1816 in 5000
Frame 1817 in 5000
Frame 1818 in 5000
Frame 1819 in 5000
Frame 1820 in 5000
Frame 1821 in 5000
Frame 1822 in 5000
Frame 1823 in 5000
Frame 1824 in 5000
Frame 1825 in 5000
Frame 1826 in 5000
Frame 1827 in 5000
Frame 1828 in 5000
Frame 1829 in 5000
Frame 1830 in 5000
Frame 1831 in 5000
Frame 1832 in 5000
Frame 1833 i

...

Frame 3077 in 5000
Frame 3078 in 5000
Frame 3079 in 5000
Frame 3080 in 5000
Frame 3081 in 5000
Frame 3082 in 5000
Frame 3083 in 5000
Frame 3084 in 5000
Frame 3085 in 5000
Frame 3086 in 5000
Frame 3087 in 5000
Frame 3088 in 5000
Frame 3089 in 5000
Frame 3090 in 5000
Frame 3091 in 5000
Frame 3092 in 5000
Frame 3093 in 5000
Frame 3094 in 5000
Frame 3095 in 5000
Frame 3096 in 5000
Frame 3097 in 5000
Frame 3098 in 5000
Frame 3099 in 5000
Frame 3100 in 5000
Frame 3101 in 5000
Frame 3102 in 5000
Frame 3103 in 5000
Frame 3104 in 5000
Frame 3105 in 5000
Frame 3106 in 5000
Frame 3107 in 5000
Frame 3108 in 5000
Frame 3109 in 5000
Frame 3110 in 5000
Frame 3111 in 5000
Frame 3112 in 5000
Frame 3113 in 5000
Frame 3114 in 5000
Frame 3115 in 5000
Frame 3116 in 5000
Frame 3117 in 5000
Frame 3118 in 5000
Frame 3119 in 5000
Frame 3120 in 5000
Frame 3121 in 5000
Frame 3122 in 5000
Frame 3123 in 5000
Frame 3124 in 5000
Frame 3125 in 5000
Frame 3126 in 5000
Frame 3127 in 5000
Frame 3128 in 5000
Frame 3129 i

Frame 3509 in 5000
Frame 3510 in 5000
Frame 3511 in 5000
Frame 3512 in 5000
Frame 3513 in 5000
Frame 3514 in 5000
Frame 3515 in 5000
Frame 3516 in 5000
Frame 3517 in 5000
Frame 3518 in 5000
Frame 3519 in 5000
Frame 3520 in 5000
Frame 3521 in 5000
Frame 3522 in 5000
Frame 3523 in 5000
Frame 3524 in 5000
Frame 3525 in 5000
Frame 3526 in 5000
Frame 3527 in 5000
Frame 3528 in 5000
Frame 3529 in 5000
Frame 3530 in 5000
Frame 3531 in 5000
Frame 3532 in 5000
Frame 3533 in 5000
Frame 3534 in 5000
Frame 3535 in 5000
Frame 3536 in 5000
Frame 3537 in 5000
Frame 3538 in 5000
Frame 3539 in 5000
Frame 3540 in 5000
Frame 3541 in 5000
Frame 3542 in 5000
Frame 3543 in 5000
Frame 3544 in 5000
Frame 3545 in 5000
Frame 3546 in 5000
Frame 3547 in 5000
Frame 3548 in 5000
Frame 3549 in 5000
Frame 3550 in 5000
Frame 3551 in 5000
Frame 3552 in 5000
Frame 3553 in 5000
Frame 3554 in 5000
Frame 3555 in 5000
Frame 3556 in 5000
Frame 3557 in 5000
Frame 3558 in 5000
Frame 3559 in 5000
Frame 3560 in 5000
Frame 3561 i

Frame 3941 in 5000
Frame 3942 in 5000
Frame 3943 in 5000
Frame 3944 in 5000
Frame 3945 in 5000
Frame 3946 in 5000
Frame 3947 in 5000
Frame 3948 in 5000
Frame 3949 in 5000
Frame 3950 in 5000
Frame 3951 in 5000
Frame 3952 in 5000
Frame 3953 in 5000
Frame 3954 in 5000
Frame 3955 in 5000
Frame 3956 in 5000
Frame 3957 in 5000
Frame 3958 in 5000
Frame 3959 in 5000
Frame 3960 in 5000
Frame 3961 in 5000
Frame 3962 in 5000
Frame 3963 in 5000
Frame 3964 in 5000
Frame 3965 in 5000
Frame 3966 in 5000
Frame 3967 in 5000
Frame 3968 in 5000
Frame 3969 in 5000
Frame 3970 in 5000
Frame 3971 in 5000
Frame 3972 in 5000
Frame 3973 in 5000
Frame 3974 in 5000
Frame 3975 in 5000
Frame 3976 in 5000
Frame 3977 in 5000
Frame 3978 in 5000
Frame 3979 in 5000
Frame 3980 in 5000
Frame 3981 in 5000
Frame 3982 in 5000
Frame 3983 in 5000
Frame 3984 in 5000
Frame 3985 in 5000
Frame 3986 in 5000
Frame 3987 in 5000
Frame 3988 in 5000
Frame 3989 in 5000
Frame 3990 in 5000
Frame 3991 in 5000
Frame 3992 in 5000
Frame 3993 i

Frame 4373 in 5000
Frame 4374 in 5000
Frame 4375 in 5000
Frame 4376 in 5000
Frame 4377 in 5000
Frame 4378 in 5000
Frame 4379 in 5000
Frame 4380 in 5000
Frame 4381 in 5000
Frame 4382 in 5000
Frame 4383 in 5000
Frame 4384 in 5000
Frame 4385 in 5000
Frame 4386 in 5000
Frame 4387 in 5000
Frame 4388 in 5000
Frame 4389 in 5000
Frame 4390 in 5000
Frame 4391 in 5000
Frame 4392 in 5000
Frame 4393 in 5000
Frame 4394 in 5000
Frame 4395 in 5000
Frame 4396 in 5000
Frame 4397 in 5000
Frame 4398 in 5000
Frame 4399 in 5000
Frame 4400 in 5000
Frame 4401 in 5000
Frame 4402 in 5000
Frame 4403 in 5000
Frame 4404 in 5000
Frame 4405 in 5000
Frame 4406 in 5000
Frame 4407 in 5000
Frame 4408 in 5000
Frame 4409 in 5000
Frame 4410 in 5000
Frame 4411 in 5000
Frame 4412 in 5000
Frame 4413 in 5000
Frame 4414 in 5000
Frame 4415 in 5000
Frame 4416 in 5000
Frame 4417 in 5000
Frame 4418 in 5000
Frame 4419 in 5000
Frame 4420 in 5000
Frame 4421 in 5000
Frame 4422 in 5000
Frame 4423 in 5000
Frame 4424 in 5000
Frame 4425 i

Frame 4805 in 5000
Frame 4806 in 5000
Frame 4807 in 5000
Frame 4808 in 5000
Frame 4809 in 5000
Frame 4810 in 5000
Frame 4811 in 5000
Frame 4812 in 5000
Frame 4813 in 5000
Frame 4814 in 5000
Frame 4815 in 5000
Frame 4816 in 5000
Frame 4817 in 5000
Frame 4818 in 5000
Frame 4819 in 5000
Frame 4820 in 5000
Frame 4821 in 5000
Frame 4822 in 5000
Frame 4823 in 5000
Frame 4824 in 5000
Frame 4825 in 5000
Frame 4826 in 5000
Frame 4827 in 5000
Frame 4828 in 5000
Frame 4829 in 5000
Frame 4830 in 5000
Frame 4831 in 5000
Frame 4832 in 5000
Frame 4833 in 5000
Frame 4834 in 5000
Frame 4835 in 5000
Frame 4836 in 5000
Frame 4837 in 5000
Frame 4838 in 5000
Frame 4839 in 5000
Frame 4840 in 5000
Frame 4841 in 5000
Frame 4842 in 5000
Frame 4843 in 5000
Frame 4844 in 5000
Frame 4845 in 5000
Frame 4846 in 5000
Frame 4847 in 5000
Frame 4848 in 5000
Frame 4849 in 5000
Frame 4850 in 5000
Frame 4851 in 5000
Frame 4852 in 5000
Frame 4853 in 5000
Frame 4854 in 5000
Frame 4855 in 5000
Frame 4856 in 5000
Frame 4857 i